In [2]:
import re
import PyPDF2
import pdfplumber
import pandas as pd
import openai
from num2words import num2words
import os
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import time

In [ ]:
input_pdf = "D:\\Code\\CognitiveSearchGPT\\backend\\data\\2022_CMB_Report.pdf"  # 请将该路径替换为实际的输入PDF路径
output_pdf_prefix = "D:\\Code\\CognitiveSearchGPT\\backend\\data\\CMB\\CMB_Report"    # 输出文件的前缀

In [ ]:
AZURE_OPENAI_API_KEY_SOUTH_CENTRAL_US=
AZURE_OPENAI_BASE=
openai.api_type = "azure"
openai.api_key = AZURE_OPENAI_API_KEY_SOUTH_CENTRAL_US
openai.api_base = AZURE_OPENAI_BASE
openai.api_version = "2022-12-01"

In [ ]:
bboxes = []

def not_within_bboxes(obj):
    """Check if the object is in any of the table's bbox."""
    def obj_in_bbox(_bbox):
        """See https://github.com/jsvine/pdfplumber/blob/stable/pdfplumber/table.py#L404"""
        v_mid = (obj["top"] + obj["bottom"]) / 2
        h_mid = (obj["x0"] + obj["x1"]) / 2
        x0, top, x1, bottom = _bbox
        return (h_mid >= x0) and (h_mid < x1) and (v_mid >= top) and (v_mid < bottom)
    return not any(obj_in_bbox(__bbox) for __bbox in bboxes)

In [ ]:
def curves_to_edges(cs):
    """See https://github.com/jsvine/pdfplumber/issues/127"""
    edges = []
    for c in cs:
        edges += pdfplumber.utils.rect_to_edges(c)
    return edges

In [ ]:
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

In [ ]:
with pdfplumber.open(input_pdf) as pdf:
        total_pages = len(pdf.pages)

        page_indices = []
        sections = []
        tables = []
        for i, page in enumerate(pdf.pages):
            page_text = page.extract_text()
            ts = {
                "vertical_strategy": "explicit",
                "horizontal_strategy": "explicit",
                "explicit_vertical_lines": curves_to_edges(page.curves + page.edges),
                "explicit_horizontal_lines": curves_to_edges(page.curves + page.edges),
                "intersection_y_tolerance": 10,
            }
            global bboxes
            bboxes = [table.bbox for table in page.find_tables(table_settings=ts)]
            page_text=page.filter(not_within_bboxes).extract_text()
            table_settings = {
                "vertical_strategy": "text",
                "horizontal_strategy": "text"
            }
            table = page.extract_tables(table_settings)
            if len(table) > 0:
                tables.append(table)
            # print(tables)
            rx = re.compile(r'''
                ^(?:Section\ )?(\d+\.+\d+)+
                [\s\S]*?
                (?=^(?:Section\ )?(\d+\.+\d+)+|\Z)

                ''', re.VERBOSE | re.MULTILINE)

            parts = [match.group(0) for match in rx.finditer(page_text)]
            # print(parts)
            for part in parts:
                if len(part) > 0:
                    sections.append(part)
            # if len(parts) > 0:
            #     sections.append(parts)
            # if find_chapters(page_text):
            #     page_indices.append(i)
        print(f"Total {len(sections)} parts.")
        print(f"Total {len(tables)} tables")

In [ ]:
table = tables[10]
df = pd.DataFrame(table[1::],columns=table[0])
df

In [ ]:
sections[15]

In [ ]:
section_embedding=get_embedding(sections[15],engine='text-embedding-ada-002')

In [ ]:
section_embedding

In [ ]:
table_embedding = get_embedding(df.to_html(),engine='text-embedding-ada-002')

In [ ]:
table_embedding

In [ ]:
tables_html=[]
for table in tables:
    df = pd.DataFrame(table[1::],columns=table[0])
    tables_html.append(df.to_html())
            
df_pdf = pd.DataFrame({'text':sections+tables_html})

In [ ]:
len(df_pdf)

In [ ]:
# Split dataframes to meet Azure OpenAI rate/token limit, may take up to 20mins
df_split = np.array_split(df_pdf,10)
for index, df in enumerate(df_split):
    if not os.path.exists(f"D:\\Code\\CognitiveSearchGPT\\backend\\data\\CMB\\CMB_Report_embedding_{index}.csv"):
        df['text']= df["text"].apply(lambda x : normalize_text(x))
        df['ada_v2'] = df['text'].apply(lambda x : get_embedding(x, engine = 'text-embedding-ada-002'))
        df.to_csv(f'D:\\Code\\CognitiveSearchGPT\\backend\\data\\CMB\\CMB_Report_embedding_{index}.csv',mode='a',header=False,encoding='utf-8')
        time.sleep(60)
# df_pdf['ada_v2'] = df_pdf["text"].apply(lambda x : get_embedding(x, engine = 'text-embedding-ada-002')) # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

In [ ]:
df = df_split[8]
df.head(4)

In [ ]:
dfs = []
for i in range(10):
    df = pd.read_csv(f"D:\\Code\\CognitiveSearchGPT\\backend\\data\\CMB\\CMB_Report_embedding_{i}.csv", names= ['id','text','ada_v2'], encoding="utf-8")
    dfs.append(df)
df_all = pd.concat(dfs)
df_all

In [ ]:
df = pd.read_csv(f"D:\\Code\\CognitiveSearchGPT\\backend\\data\\CMB\\CMB_Report_embedding_{0}.csv",names= ['id','text','ada_v2'], encoding="utf-8")
df.head(5)
df.info()

In [ ]:
# search through the reviews for a specific product
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        engine="text-embedding-ada-002" # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df["ada_v2"].apply(lambda x: cosine_similarity(eval(x), embedding))

    res = (
            df.sort_values("similarities", ascending=False)
            .head(top_n)
        )
    return res


res = search_docs(df_all, "2022年总资产是多少?", top_n=4)

In [ ]:
res

In [ ]:
question = "给我财务业绩摘要表"
res = search_docs(df_all, question, top_n=4)

context = res['text'].values.tolist()
context_str = "\n".join(context)
prompt = '''<|im_start|>系统助理帮助公司员工解决他们的问题，回答要简短。
仅回答以下来源列表中列出的事实。如果下面没有足够的信息，请说您不知道。不要生成不使用以下来源的答案。不要使用年代久远的来源信息。对于表格形式的数据，请以HTML表格形式输出，不要使用Markdown表格.
来源:
{sources}
<|im_end|>
<|im_start|>
user: {question}
<|im_end|>
<|im_start|>assistant:
'''
completion = openai.Completion.create(
            engine='gpt-35-turbo', 
            prompt=prompt.format(sources=context_str,question=question), 
            temperature=0.0, 
            max_tokens=2000, 
            n=1, 
            stop=["<|im_end|>", "<|im_start|>"])

result = completion.choices[0].text
print(result)

In [3]:
openai.api_type = "azure"
openai.api_key = '4f1040b5daad408abe969ceb1f82b6df'
openai.api_base = 'https://futuopenaius01.openai.azure.com/'
openai.api_version = "2022-12-01"

question = '给我写一个关于富途上线GPT产品的广告语，要面向金融客户，要有激情'
completion = openai.Completion.create(
            engine='gpt-35-turbo', 
            prompt='''<|im_start|>系统助理帮助公司员工解决他们的问题，回答要简短。<|im_end|><|im_start|>user:{question}<|im_end|><|im_start>assistant:'''.format(question=question), 
            temperature=0.7, 
            max_tokens=2000, 
            n=1, 
            stop=["<|im_end|>", "<|im_start|>"])

InvalidRequestError: The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.